In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer

In [2]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [3]:
from sklearn.impute import KNNImputer

In [4]:
df_bank = pd.read_csv("datos/bank-additional-full-edit4.csv", index_col= 0)
df_bank.head(2)

,job,marital,education,default,housing,loan,contact,poutcome,y,month_day_week,...,day_week,age,duration,campaign,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,housemaid,married,basic.4y,0.0,0.0,0.0,telephone,NONEXISTENT,no,"['may', 'mon']",...,'mon'],56,261,1,0,1.1,93.994,-36.4,4.857,5191.0
1,services,married,high.school,NaN,0.0,0.0,telephone,NONEXISTENT,no,"['may', 'mon']",...,'mon'],57,149,1,0,1.1,93.994,-36.4,4.857,5191.0


In [5]:
df_bank.dtypes

job                object
marital            object
education          object
default           float64
housing           float64
loan              float64
contact            object
poutcome           object
y                  object
month_day_week     object
loan2              object
housing2           object
default2           object
education2         object
job2               object
month              object
day_week           object
age                 int64
duration            int64
campaign            int64
previous            int64
emp_var_rate      float64
cons_price_idx    float64
cons_conf_idx     float64
euribor3m         float64
nr_employed       float64
dtype: object

Es el momento de eliminar los nulos:

- Reemplazad los valores nulos del resto de las columnas categóricas por la moda, usando el método SimpleImputer.
💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

- Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [6]:
df_numericas = df_bank.select_dtypes(include= np.number)

In [7]:
df_numericas.dtypes

default           float64
housing           float64
loan              float64
age                 int64
duration            int64
campaign            int64
previous            int64
emp_var_rate      float64
cons_price_idx    float64
cons_conf_idx     float64
euribor3m         float64
nr_employed       float64
dtype: object

In [8]:
def eliminar_nulos_iterative(dataframe_completo):
    dataframe = dataframe_completo.select_dtypes(include = np.number)
    imputer = IterativeImputer(n_nearest_features=None, imputation_order='ascending')
    imputer.fit(dataframe)
    global numericas_trans
    numericas_trans = pd.DataFrame(imputer.transform(dataframe), columns = dataframe.columns)


    dataframe_completo2 = dataframe_completo.drop(dataframe.columns, axis = 1)
    dataframe_completo2 = pd.concat([dataframe_completo2, numericas_trans], axis = 1)
    
    return dataframe_completo2

In [9]:
df_bank_sin_nulos_it = eliminar_nulos_iterative(df_bank)

In [10]:
df_bank_sin_nulos_it.head()

,job,marital,education,contact,poutcome,y,month_day_week,loan2,housing2,default2,...,loan,age,duration,campaign,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,housemaid,married,basic.4y,telephone,NONEXISTENT,no,"['may', 'mon']",No,No,No,...,0.0,56.0,261.0,1.0,0.0,1.1,93.994,-36.4,4.857,5191.0
1,services,married,high.school,telephone,NONEXISTENT,no,"['may', 'mon']",No,No,unknown,...,0.0,57.0,149.0,1.0,0.0,1.1,93.994,-36.4,4.857,5191.0
2,services,married,high.school,telephone,NONEXISTENT,no,"['may', 'mon']",No,Si,No,...,0.0,37.0,226.0,1.0,0.0,1.1,93.994,-36.4,4.857,5191.0
3,admin.,married,basic.6y,telephone,NONEXISTENT,no,"['may', 'mon']",No,No,No,...,0.0,40.0,151.0,1.0,0.0,1.1,93.994,-36.4,4.857,5191.0
4,services,married,high.school,telephone,NONEXISTENT,no,"['may', 'mon']",Si,No,No,...,1.0,56.0,307.0,1.0,0.0,1.1,93.994,-36.4,4.857,5191.0


In [11]:
df_bank_sin_nulos_it.isnull().sum()

job                330
marital              0
education         1730
contact              0
poutcome             0
y                    0
month_day_week       0
loan2                0
housing2             0
default2             0
education2           0
job2                 0
month                0
day_week             0
default              0
housing              0
loan                 0
age                  0
duration             0
campaign             0
previous             0
emp_var_rate         0
cons_price_idx       0
cons_conf_idx        0
euribor3m            0
nr_employed          0
dtype: int64

- Reemplazad los valores nulos de la columna age por la media de la edad usando el método KNN Imputer.

In [12]:
df_bank.head()

,job,marital,education,default,housing,loan,contact,poutcome,y,month_day_week,...,day_week,age,duration,campaign,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,housemaid,married,basic.4y,0.0,0.0,0.0,telephone,NONEXISTENT,no,"['may', 'mon']",...,'mon'],56,261,1,0,1.1,93.994,-36.4,4.857,5191.0
1,services,married,high.school,NaN,0.0,0.0,telephone,NONEXISTENT,no,"['may', 'mon']",...,'mon'],57,149,1,0,1.1,93.994,-36.4,4.857,5191.0
2,services,married,high.school,0.0,1.0,0.0,telephone,NONEXISTENT,no,"['may', 'mon']",...,'mon'],37,226,1,0,1.1,93.994,-36.4,4.857,5191.0
3,admin.,married,basic.6y,0.0,0.0,0.0,telephone,NONEXISTENT,no,"['may', 'mon']",...,'mon'],40,151,1,0,1.1,93.994,-36.4,4.857,5191.0
4,services,married,high.school,0.0,0.0,1.0,telephone,NONEXISTENT,no,"['may', 'mon']",...,'mon'],56,307,1,0,1.1,93.994,-36.4,4.857,5191.0


In [13]:
def reemplazar_nulos_knn(dataframe, dataframe_completo):
    imputerKNN = KNNImputer(n_neighbors=5)
    imputerKNN.fit(dataframe)
    transform_knn = imputerKNN.transform(dataframe)
    numericas_trans = pd.DataFrame(transform_knn, columns = dataframe.columns)

    dataframe_completo.drop(dataframe.columns, axis = 1, inplace= True)
    dataframe_completo = pd.concat([dataframe_completo, numericas_trans], axis = 1)
    
    return dataframe_completo

In [14]:
df_bank_reemplazar_knn = reemplazar_nulos_knn(df_numericas, df_bank)

In [15]:
df_bank_reemplazar_knn.head()

,job,marital,education,contact,poutcome,y,month_day_week,loan2,housing2,default2,...,loan,age,duration,campaign,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,housemaid,married,basic.4y,telephone,NONEXISTENT,no,"['may', 'mon']",No,No,No,...,0.0,56.0,261.0,1.0,0.0,1.1,93.994,-36.4,4.857,5191.0
1,services,married,high.school,telephone,NONEXISTENT,no,"['may', 'mon']",No,No,unknown,...,0.0,57.0,149.0,1.0,0.0,1.1,93.994,-36.4,4.857,5191.0
2,services,married,high.school,telephone,NONEXISTENT,no,"['may', 'mon']",No,Si,No,...,0.0,37.0,226.0,1.0,0.0,1.1,93.994,-36.4,4.857,5191.0
3,admin.,married,basic.6y,telephone,NONEXISTENT,no,"['may', 'mon']",No,No,No,...,0.0,40.0,151.0,1.0,0.0,1.1,93.994,-36.4,4.857,5191.0
4,services,married,high.school,telephone,NONEXISTENT,no,"['may', 'mon']",Si,No,No,...,1.0,56.0,307.0,1.0,0.0,1.1,93.994,-36.4,4.857,5191.0


In [16]:
df_bank_reemplazar_knn.isnull().sum()

job                330
marital              0
education         1730
contact              0
poutcome             0
y                    0
month_day_week       0
loan2                0
housing2             0
default2             0
education2           0
job2                 0
month                0
day_week             0
default              0
housing              0
loan                 0
age                  0
duration             0
campaign             0
previous             0
emp_var_rate         0
cons_price_idx       0
cons_conf_idx        0
euribor3m            0
nr_employed          0
dtype: int64

- Reemplazad los valores nulos de la columna age por la media de la edad usando el método Simple Imputer.

- ¿Podríais explicar qué diferencia hay entre estos últimos tres ejercicios?

Guardad el csv.